In [ ]:
!pip install pandas textblob dash dash-core-components dash-html-components dash-bootstrap-components plotly

In [ ]:
!pip install jupyter-dash

In [16]:
import pandas as pd
from textblob import TextBlob
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [39]:
# Load the data from the CSV file
data = pd.read_csv('/content/JLI21080077.csv')

Sentiment analysis using TextBlob

In [40]:
# All values in 'Survey Responses Survey Comments C' are strings
data['Survey Responses Survey Comments C'] = data['Survey Responses Survey Comments C'].fillna('').astype(str)

# Perform sentiment analysis and label the sentiments
def get_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

data['Sentiment'] = data['Survey Responses Survey Comments C'].apply(get_sentiment)

# Label the sentiments
def label_sentiment(score):
    if score > 0:
        return 'Positive'
    elif score < 0:
        return 'Negative'
    else:
        return 'Neutral'

data['Sentiment_Label'] = data['Sentiment'].apply(label_sentiment)

Sentiment Analysis using Vader

In [21]:
# All values in 'Survey Responses Survey Comments C' are strings
data['Survey Responses Survey Comments C'] = data['Survey Responses Survey Comments C'].fillna('').astype(str)

# Initialize the VADER sentiment intensity analyzer
sid = SentimentIntensityAnalyzer()

# Perform sentiment analysis and label the sentiments
def get_sentiment(text):
    scores = sid.polarity_scores(text)
    return scores['compound']

data['Sentiment'] = data['Survey Responses Survey Comments C'].apply(get_sentiment)

# Label the sentiments based on compound score
def label_sentiment(score):
    if score > 0.05:
        return 'Positive'
    elif score < -0.05:
        return 'Negative'
    else:
        return 'Neutral'

data['Sentiment_Label'] = data['Sentiment'].apply(label_sentiment)

In [41]:
# Display the DataFrame to ensure it's loaded correctly
data.head()

,Survey Responses ID,Survey Responses Case Number C,Survey Responses Case Number,Survey Responses Survey Score C,Survey Responses Survey Comments C,Survey Responses Case Created Week,Survey Responses Case Status,Survey Responses Last Agent Owner C,Survey Responses Case Description,Survey Responses Case Topic,...,Survey Responses Origin,Survey Responses Resolution,Survey Responses First Response Time In Hours,Survey Responses Resolution Time to Solved,Survey Responses How Satisfied Support Juulvapor Com,Survey Responses Would You Speak to This Care Team Member,Survey Responses How Well Did We Answer Your Questions,Survey Responses How Satisfied Are You with Your Product,Sentiment,Sentiment_Label
0,a4639000003EcgMAAS,50039000026EbSEAA0,2396831,1,I have purchased 5 juuls in the past year. The...,6/24/18,Solved,Thed Africa,Still have not received a tracking number. Als...,Shipping,...,Chat,Resolved with Troubleshooting,0.0,1,1.0,No,2.0,3.0,0.014502,Positive
1,a4639000003Ec1xAAC,50039000026DSHuAAO,2390020,1,"My order was never delivered, was promised a $...",6/24/18,Solved,Ginalyn Villalon,"My order wasnt delivered, just got it reordere...",Shipping,...,Chat,Resolved with Troubleshooting,1.0,1,1.0,No,1.0,1.0,0.000000,Neutral
2,a4639000003Eci8AAC,50039000026ESzgAAG,2395899,1,Not Happy at all .... warranty replacement was...,6/24/18,Closed,NaN,Nothing else. Device not working,Device,...,Support Form,Coupon Code Sent,NaN,0,1.0,No,1.0,1.0,-0.500000,Negative
3,a4639000003Eca9AAC,50039000026E7ElAAK,2393773,1,juul,6/24/18,Solved,Leslie V,NaN,Shipping,...,Support Form,Resolved with Troubleshooting,0.0,0,2.0,No,2.0,1.0,0.000000,Neutral
4,a4639000003EcQ9AAK,50039000026EE3BAAW,2394493,1,So i legally able to buy my juul at 18 but can...,6/24/18,Closed,NaN,Does not hit hit at all,Device,...,Support Form,Coupon Code Sent,NaN,0,1.0,No,1.0,1.0,0.500000,Positive


Visualize

In [42]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Function to filter data based on selected options
def filter_data(data, selected_topics, selected_issues, selected_reasons):
    filtered_data = data.copy()

    if selected_topics:
        filtered_data = filtered_data[filtered_data['Survey Responses Case Topic'].isin(selected_topics)]

    if selected_issues:
        filtered_data = filtered_data[filtered_data['Survey Responses Case Issue'].isin(selected_issues)]

    if selected_reasons:
        filtered_data = filtered_data[filtered_data['Survey Responses Case Reason'].isin(selected_reasons)]

    return filtered_data

# Initial filter selections (you can modify these)
selected_topics = data['Survey Responses Case Topic'].unique().tolist()
selected_issues = data['Survey Responses Case Issue'].unique().tolist()
selected_reasons = data['Survey Responses Case Reason'].unique().tolist()

# Filter data based on initial selections
filtered_data = filter_data(data, selected_topics, selected_issues, selected_reasons)

# Count the number of each sentiment label
sentiment_counts = filtered_data['Sentiment_Label'].value_counts().reset_index()
sentiment_counts.columns = ['Sentiment', 'Count']

# Ensure all sentiment categories are present
sentiment_categories = ['Positive', 'Neutral', 'Negative']
for category in sentiment_categories:
    if category not in sentiment_counts['Sentiment'].values:
        sentiment_counts = sentiment_counts.append({'Sentiment': category, 'Count': 0}, ignore_index=True)

# Sort sentiment categories
sentiment_counts = sentiment_counts.set_index('Sentiment').loc[sentiment_categories].reset_index()

# Create the divergent bar chart
fig = make_subplots(rows=1, cols=1)

colors = {'Positive': 'green', 'Neutral': 'gray', 'Negative': 'red'}
for sentiment in sentiment_categories:
    count = sentiment_counts[sentiment_counts['Sentiment'] == sentiment]['Count'].values[0]
    fig.add_trace(
        go.Bar(
            x=[count if sentiment != 'Negative' else -count],
            y=[sentiment],
            orientation='h',
            name=sentiment,
            marker=dict(color=colors[sentiment])
        )
    )

fig.update_layout(
    title='Sentiment Analysis of Survey Comments',
    barmode='relative',
    xaxis=dict(title='Number of Comments'),
    yaxis=dict(title='Sentiment')
)

fig.show()

Run the code cell below to visualize an interactive network graph where each node represents a case, color-coded by sentiment, and the hover text shows the case number, sentiment, and case topic.

This helps to identify clusters of cases that share the same sentiment, providing preliminary insights into common patterns or issues in the feedback.

In [43]:
import networkx as nx
import plotly.graph_objects as go

# Define a color map for sentiment labels
color_map = {'Positive': 'green','Neutral': 'gray','Negative': 'red'}

# Function to create network graph
def create_network_graph(data):
    G = nx.Graph()

    # Add nodes
    for index, row in data.iterrows():
        G.add_node(row['Survey Responses Case Number'], sentiment=row['Sentiment_Label'], topic=row['Survey Responses Case Topic'])

    # Add edges between nodes with the same sentiment
    for i, row_i in data.iterrows():
        for j, row_j in data.iterrows():
            if i < j:
                if row_i['Sentiment_Label'] == row_j['Sentiment_Label']:
                    G.add_edge(row_i['Survey Responses Case Number'], row_j['Survey Responses Case Number'])

    pos = nx.spring_layout(G)

    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    node_x = []
    node_y = []
    node_color = []
    node_text = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_color.append(color_map[G.nodes[node]['sentiment']])
        node_text.append(f'Case Number: {node}<br>Sentiment: {G.nodes[node]["sentiment"]}<br>Topic: {G.nodes[node]["topic"]}')

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        text=node_text,
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            size=10,
            color=node_color,
            colorbar=dict(
                thickness=15,
                title='Node Sentiment',
                xanchor='left',
                titleside='right'
            )
        )
    )

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='Network Graph of Sentiment Analysis',
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20,l=5,r=5,t=40)
                    ))
    return fig

# Create and show the network graph
fig = create_network_graph(data)
fig.show()